# Aprendizem Profunda - DNN
### Tarefa III
1. Rúben Gonçalo Araújo da Silva pg57900   
2. José Luis Fraga Costa pg55970
3. Pedro Miguel Costa Azevedo pg57897
4. Rui Pedro Fernandes Madeira Pinto pg56010

# Implementação

### imports

1. pandas
2. tensorflow
3. sklearn

In [215]:
# Standard Library
import os
import csv

# Data Handling
import pandas as pd
import numpy as np


# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, classification_report

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, Flatten, Dense, Dropout, SimpleRNN, GRU, LSTM, 
    Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, 
    BatchNormalization, Conv1D, MaxPooling1D, LayerNormalization, Add, Input
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, AdamW

# Hugging Face Transformers
from transformers import (
    AutoTokenizer, 
    TFAutoModelForSequenceClassification, 
    pipeline
)


### Input dos dados

In [216]:
df = pd.read_csv("data/fixed_dataset_final.csv")

df = df.dropna(subset=["Label"])
df["Text"] = df["Text"].fillna("")
all_ids = df["ID"].fillna("")
test_ids = df["ID"].fillna("")

# Checar balanceamento das classes
print(df['Label'].value_counts())

# Verificar duplicados
print("Duplicados:", df.duplicated(subset='Text').sum())

# Exibir amostra dos dados
print(df.head())



Label
Human    2100
AI       1953
Name: count, dtype: int64
Duplicados: 2
     ID                                               Text  Label
0  D1-1  Advanced electromagnetic potentials are indige...  Human
1  D1-2  This research paper investigates the question ...     AI
2  D1-3  We give an algorithm for finding network encod...  Human
3  D1-4  The paper presents an efficient centralized bi...     AI
4  D1-5  We introduce an exponential random graph model...  Human


### Função pra gravar em csv

In [217]:
def to_csv(results_df, name):
    os.makedirs("score", exist_ok=True)
    path = f"score/{name}.csv"
    filtered_df = results_df[results_df["ID"].notna() & (results_df["ID"] != "")].copy()
    filtered_df['prefix'] = filtered_df['ID'].str.extract(r'(D[0-9])')
    filtered_df['number'] = filtered_df['ID'].str.extract(r'-(\d+)').astype(int)
    filtered_df = filtered_df.sort_values(by=['prefix', 'number'])

    with open(path, 'w') as f:
        f.write("ID\tLabel\n") 
        for index, row in filtered_df.iterrows():
            f.write(f"{row['ID']}\t{row['Label']}\n") 

In [218]:
def predict_and_save(model, X_data, ids, model_name):
    # Generate predictions
    y_pred = (model.predict(X_data) > 0.5).astype(int).flatten()

    # Convert numeric predictions to labels
    labels = ["Human" if label == 0 else "AI" for label in y_pred]
    # Create results DataFrame
    results_df = pd.DataFrame({
        "ID": ids,
        "Label": labels
    })
    to_csv(results_df, model_name)

### Train Test Split (divisão de dados)

In [219]:
# Remove duplicates by keeping the first occurrence
df_unique = df.drop_duplicates(subset=["Text"], keep="first")

# Check how many rows remain
print(f"Original dataset size: {len(df)}, Unique texts: {len(df_unique)}")

# Split the deduplicated dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_unique["Text"], df_unique["Label"], test_size=0.2, random_state=42, stratify=df_unique["Label"]
)
test_indices = test_texts.index

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

# Verify no overlap
print(f"Overlap between train and test after deduplication: {len(set(train_texts) & set(test_texts))}")


Original dataset size: 4053, Unique texts: 4051
Overlap between train and test after deduplication: 0


### Tokenizer

In [220]:
# Tokenize the data
tokenizer = Tokenizer(num_words=25000)
tokenizer.fit_on_texts(train_texts)

X_train_seq = tokenizer.texts_to_sequences(train_texts)
X_val_seq = tokenizer.texts_to_sequences(val_texts)
X_test_seq = tokenizer.texts_to_sequences(test_texts)

X_train = pad_sequences(X_train_seq, maxlen=200)
X_val = pad_sequences(X_val_seq, maxlen=200)
X_test = pad_sequences(X_test_seq, maxlen=200)

# Convert string labels to numeric values
label_map = {"Human": 0, "AI": 1}  # Define mapping
y_train = np.array([label_map[label] for label in train_labels], dtype=np.float32)
y_val = np.array([label_map[label] for label in val_labels], dtype=np.float32)
y_test = np.array([label_map[label] for label in test_labels], dtype=np.float32)

## Dataset de Avaliação

In [221]:
data_set_professor = pd.read_csv("data/dataset1_inputs.csv", sep='\t', on_bad_lines='skip')
data_set_professor["Text"] = data_set_professor["Text"].fillna("")
X_professor_seq = tokenizer.texts_to_sequences(data_set_professor["Text"])
X_professor = pad_sequences(X_professor_seq, maxlen=100)  

### Early Stopping

In [222]:
early_stopping = EarlyStopping(
    monitor='val_loss',     
    patience=5,            
    restore_best_weights=True,           
)

### Reduce_lr

In [223]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5, 
    patience=3,
    min_lr=0.00005
)

## CNN

In [224]:
model_cnn = Sequential([
    Embedding(input_dim=25000, output_dim=32, input_length=200),

    tf.keras.layers.GaussianNoise(0.05),
    
    Conv1D(filters=32, kernel_size=5, activation=tf.keras.layers.LeakyReLU(alpha=0.1), 
           kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    
    GlobalMaxPooling1D(),
    Dropout(0.5),
    
    Dense(16, activation=tf.keras.layers.LeakyReLU(alpha=0.1), kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    
    Dense(1, activation='sigmoid')
])

# Compile with AdamW and gradient clipping
initial_learning_rate = 0.0008
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=5 * len(X_train) // 128, decay_rate=0.9, staircase=True
)
model_cnn.compile(optimizer=AdamW(learning_rate=lr_schedule, weight_decay=0.01, clipnorm=1.0),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

# Train the model
history = model_cnn.fit(
    X_train, y_train,
    epochs=25,
    batch_size=128,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/25


c:\Users\ruben\Desktop\Minho\MEI\SI\AP\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
c:\Users\ruben\Desktop\Minho\MEI\SI\AP\.venv\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - accuracy: 0.5302 - loss: 5.2653 - val_accuracy: 0.4815 - val_loss: 1.2043 - learning_rate: 8.0000e-04
Epoch 2/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.5066 - loss: 4.0945 - val_accuracy: 0.4815 - val_loss: 1.1882 - learning_rate: 8.0000e-04
Epoch 3/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.4959 - loss: 3.1952 - val_accuracy: 0.4815 - val_loss: 1.1722 - learning_rate: 8.0000e-04
Epoch 4/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.4928 - loss: 2.1895 - val_accuracy: 0.4815 - val_loss: 1.1550 - learning_rate: 8.0000e-04
Epoch 5/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.4923 - loss: 1.7767 - val_accuracy: 0.4815 - val_loss: 1.1358 - learning_rate: 7.2000e-04
Epoch 6/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.5318 - loss: 1.5009 - val_accuracy: 0.4815 - val_loss: 1.1163 - learning_rate: 7.2000e-04
Epoch 7/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.5036 - loss: 1.4

## Simple RNN

# Correr Modelos

**Test Set**

In [225]:
test_ids = df.loc[test_indices, "ID"]  # Get IDs for test split only
predict_and_save(model_cnn, X_test, test_ids, "cnn")


26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


**Avaliação**

In [226]:
predict_and_save(model_cnn, X_professor, data_set_professor["ID"], "cnn_dataset1")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step


## Classification Reports

In [227]:
print(classification_report(y_test, model_cnn.predict(X_test) > 0.5))

18/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
              precision    recall  f1-score   support

         0.0       0.94      0.99      0.96       420
         1.0       0.98      0.93      0.96       391

    accuracy                           0.96       811
   macro avg       0.96      0.96      0.96       811
weighted avg       0.96      0.96      0.96       811



In [228]:
# 1. Carregar os arquivos
preds_df = pd.read_csv("score/cnn_dataset1.csv", sep='\t', on_bad_lines='skip')
labels_df = pd.read_csv("data/dataset1_outputs.csv", sep='\t', on_bad_lines='skip')

# 2. Garantir que as colunas corretas estão sendo usadas
# Assumindo que a coluna de previsões se chama 'Pred' e a de verdade se chama 'Label'
y_pred = preds_df['Label']
y_true = labels_df['Label']

# 3. Comparar previsões com os rótulos reais
print("Acurácia:", accuracy_score(y_true, y_pred))
print("Relatório de Classificação:")
print(classification_report(y_true, y_pred))

Acurácia: 0.5666666666666667
Relatório de Classificação:
              precision    recall  f1-score   support

          AI       0.75      0.20      0.32        15
       Human       0.54      0.93      0.68        15

    accuracy                           0.57        30
   macro avg       0.64      0.57      0.50        30
weighted avg       0.64      0.57      0.50        30

